In [ ]:
!sudo pip install diffusers transformers -q

In [4]:
!sudo pip install diffusers transformers -q

In [5]:

from pathlib import Path
import tqdm
import torch
import pandas as pd
import numpy as np
from diffusers import StableDiffusionPipeline
from transformers import pipeline, set_seed
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import cv2

There was a problem when trying to write in your cache folder (/home/mosaic-ai/.cache/huggingface/hub). Please, ensure the directory exists and can be written to.


RuntimeError: Failed to import diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion because of the following error (look up to see its traceback):
tokenizers>=0.11.1,!=0.11.3,<0.14 is required for a normal functioning of this module, but found tokenizers==0.14.1.
Try: pip install transformers -U or pip install -e '.[dev]' if you're working with git main